# Convert — Run a simple conversion of a PDB to the QDXF format

See [the tutorial.](../convert.ipynb)

In [1]:
import os
from glob import glob
from pathlib import Path

from pdbtools import pdb_fetch

import rush

# Define our project information
DESCRIPTION = "rush-py batch notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "convert"]
WORK_DIR = Path.home() / "qdx" / "convert"

# Set our inputs
PROTEIN_PDB_FOLDER_PATH = WORK_DIR / "proteins"

os.makedirs(PROTEIN_PDB_FOLDER_PATH, exist_ok=True)

# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(
    workspace=WORK_DIR,
    batch_tags=TAGS,
)

# Fetch pdbs
pdb_ids = ["1B39", "1GIH"]
for pdb_id in pdb_ids:
    complex = list(pdb_fetch.fetch_structure(pdb_id))
    with open(PROTEIN_PDB_FOLDER_PATH / f"{pdb_id}.pdb", "w") as f:
        for line in complex:
            f.write(line)


# Convert each pdb
for pdb in glob(str(PROTEIN_PDB_FOLDER_PATH / "*.pdb")):
    (conformer,) = await client.convert("PDB", Path(pdb), target="NIX")
    print(await conformer.get())

await client.status()